# Importando bibliotecas

In [ ]:
import pandas as pd
import numpy as np
import re
import datetime

# Importando o Dataset e limpando os dados

In [ ]:
shark_attack = pd.read_csv('data/attacks.csv', encoding = 'latin1')

Obtendo informações das colunas

In [ ]:
shark_attack.info()

Verificando as colunas com poucas informações

In [ ]:
print(shark_attack[shark_attack['Unnamed: 22'].notna()])
print(shark_attack[shark_attack['Unnamed: 23'].notna()])

Tirando as ultimas duas colunas que não possuiam informação

In [ ]:
filter1 = shark_attack.drop(columns = ['Unnamed: 22','Unnamed: 23'])
filter1

Tirar todas as linhas que possuem somente NaN

In [ ]:
filter2 = filter1.dropna(how = 'all')

Procurando os 10 paises com mais casos

In [ ]:
pais_e_data = filter2.groupby(by=['Country']).count()
pais_e_data = pais_e_data.sort_values('Case Number', ascending=False).head(10)

Tirar linhas que possuem menos do que 60% das colunas completas

In [ ]:
q60=len(shark_attack.columns)*0.6 
filter3 = filter2.dropna(thresh=q60)

Tirando palavras da coluna de data

In [ ]:
filter3['Date'] = filter3['Date'].str.replace(r'[A-z]+ ','')

Verificando as linhas com o ano sem informação

In [ ]:
filter3[filter3['Year'].isna()]

Preenchendo o ano de linhas que possuiam data mas no ano estava NaN

In [ ]:
filter3['Year'][6079]=1836
filter3['Year'][187]=2017

Limpando as informaçõs da coluna Date e criando uma nova coluna com os anos a partir destas

In [ ]:
filter3['year_new']= [re.findall(r'[0-9]{4}',data) for data in filter3['Date']]
filter3['year_new'] = [''.join(ano[-4:]) for ano in  filter3['year_new']]
filter3['year_new'] = [ano[-4:] for ano in  filter3['year_new']]

Transformando a coluna do Ano em inteiros e verificando se ainda possuiam valores não preenchidos ou errados. Vai dar erro ao rodar o codigo e a partir deste erro, é feita a correção nas seguintes linhas do código

In [ ]:
filter3['year_new'].astype(int)

Olhando as linhas nas quais os valores da coluna do Ano são vazias 

In [ ]:
filter3[filter3['year_new']=='']

Preenchendo manualmente as colunas que possuiam a informação do ano porém em outra coluna

In [ ]:
filter3['year_new'][4644]=1944
filter3['year_new'][4802]=1939
filter3['year_new'][4838]=1937

Tirando as colunas que não possuiam informações suficientes para preencher o Ano

In [ ]:
filter4= filter3.drop(filter3[filter3['year_new']==''].index)

Transformando a coluna do Ano em inteiros e verificando se ainda possuiam valores não preenchidos ou errados

In [ ]:
filter4['year_new'] = filter4['year_new'].astype(int)

Criando um dataframe so com os casos no EUA porque é o pais com mais relatos

In [ ]:
dataframe_USA = filter4[filter4['Country'].eq('USA')]

Filtrar acima do ano de 1950

In [ ]:
dataframe_USA_gt1950 =dataframe_USA[dataframe_USA['year_new']>1950]
dataframe_USA_gt1950

Verificando o somátorio de casos por ano

In [ ]:
dataframe_USA_gt1950_year = dataframe_USA_gt1950.groupby(by=['year_new']).count()
dataframe_USA_gt1950_year
sum(dataframe_USA_gt1950_year['Case Number'])

Criando novo dataframe para verificar o maior numero de casos por ano

In [ ]:
dataframe_USA_filtrado = dataframe_USA_gt1950.loc[:,['year_new','Country','Area','Activity']]
dataframe_USA_groupby = dataframe_USA_filtrado.groupby(by=['year_new']).count()
dataframe_USA_groupby['Casos'] = np.where(dataframe_USA_groupby['Country']>dataframe_USA_groupby['Area'],dataframe_USA_groupby['Country'],np.where(dataframe_USA_groupby['Area']>dataframe_USA_groupby['Activity'],dataframe_USA_groupby['Area'],dataframe_USA_groupby['Activity']))
dataframe_USA_groupby = dataframe_USA_groupby.reset_index()

Criando um segundo dataframe para conseguir fazer operação entre eles e obter a taxa por ano

In [ ]:
dataframe_USA_gt1949 =dataframe_USA[dataframe_USA['year_new']<=2017]
dataframe_USA_gt1949 =dataframe_USA_gt1949[dataframe_USA_gt1949['year_new']>1949]
dataframe_USA_filtrado2 = dataframe_USA_gt1949.loc[:,['year_new','Country','Area','Activity']]
dataframe_USA_groupby2 = dataframe_USA_filtrado2.groupby(by=['year_new']).count()
dataframe_USA_groupby2['Casos'] = np.where(dataframe_USA_groupby2['Country']>dataframe_USA_groupby2['Area'],dataframe_USA_groupby2['Country'],np.where(dataframe_USA_groupby2['Area']>dataframe_USA_groupby2['Activity'],dataframe_USA_groupby2['Area'],dataframe_USA_groupby2['Activity']))
dataframe_USA_groupby2 = dataframe_USA_groupby2.reset_index()

Criando a coluna da taxa de crescimento de ataques de tubarao nos EUA

In [ ]:
dataframe_USA_groupby['Variação Ano a Ano dos Ataques de Tubarão']= (dataframe_USA_groupby['Casos'] - dataframe_USA_groupby2['Casos'])/dataframe_USA_groupby2['Casos']

In [ ]:
dataframe_USA_groupby

In [ ]:
dataframe_USA_groupby.plot(x='year_new',y= ['Casos'])

Obtendo o primeiro gráfico para visualizar melhor os resultados

In [ ]:
dataframe_USA_groupby.plot(x='year_new',y= ['Variação Ano a Ano dos Ataques de Tubarão']) 

Limpando e renomeando as colunas

In [ ]:
dataframe_USA_groupby['Ano'] = dataframe_USA_groupby['year_new']
dataframe_USA_groupby_final = dataframe_USA_groupby.drop(columns = ['Country','Activity','Area','Ano'])
dataframe_USA_groupby_final = dataframe_USA_groupby_final.rename(columns={'year_new':'Ano'})

Dataframe final baseado no numero de casos e a taxa de crescimento dos ataques

In [ ]:
dataframe_USA_groupby_final

# Importando e selecionando o numero de linhas do dataframe com a taxa de crescimento da populaçao dos EUA

In [ ]:
crescimento_pop_usa = pd.read_excel('data/populacaoUSA.xlsx')
crescimento_pop_usa1 =crescimento_pop_usa.drop(crescimento_pop_usa.index[[68,69,70]]) 
crescimento_pop_usa1

Adicionando a coluna da taxa de crescimento da populaçao americana ao dataframe com as informações dos ataques

In [ ]:
dataframe_USA_groupby_final['Taxa de crescimento populacao EUA']= crescimento_pop_usa1[['Taxa de crescimento']]
dataframe_USA_groupby_final

Limpando a coluna com os dados da taxa de crescimento da população americana

In [ ]:
dataframe_USA_groupby_final['Taxa de crescimento populacao EUA'] = dataframe_USA_groupby_final['Taxa de crescimento populacao EUA'].str.replace(r' %','')

Preenchendo a primeira linha da coluna da taxa de crescimento dos EUA com base em busca na intenet

In [ ]:
dataframe_USA_groupby_final['Taxa de crescimento populacao EUA'][0] = 1.29

Transformando a coluna da taxa de crescimento dos EUA em float

In [ ]:
dataframe_USA_groupby_final[['Taxa de crescimento populacao EUA']] = dataframe_USA_groupby_final[['Taxa de crescimento populacao EUA']].astype(float)

In [ ]:
dataframe_USA_groupby_final

Plotando o gráfico comparando as taxas de crescimento de ataques e da populaçao dos EUA 

In [ ]:
dataframe_USA_groupby_final.plot(x='Ano',y= ['Variação Ano a Ano dos Ataques de Tubarão','Taxa de crescimento populacao EUA']) 

In [ ]:
dataframe_USA_groupby_final.plot.scatter(x ='Variação Ano a Ano dos Ataques de Tubarão' ,y='Taxa de crescimento populacao EUA', xlim=(-1,0.5), ylim=(0.6,1.3))

# Importando o dataframe com as informações do nivel do mar ao longo dos anos

In [ ]:
sea_level = pd.read_csv('data/sea-level_fig-1.csv')

Limpando o dataframe do nivel do mar

In [ ]:
sea_level['Elevação do nivel do mar(polegadas)'] = np.where(sea_level[['CSIRO - Adjusted sea level (inches)']].notna(),sea_level[['CSIRO - Adjusted sea level (inches)']],sea_level[['NOAA - Adjusted sea level (inches)']])

Criando primeiro dataframe para calculo da taxa da elevação do nivel do mar 

In [ ]:
sea_level1 =sea_level[sea_level['Year']>1949]
sea_level1= sea_level1[sea_level['Year']<2018].reset_index()
sea_level1

Criando segundo dataframe para calculo da taxa da elevação do nivel do mar

In [ ]:
sea_level2 =sea_level[sea_level['Year']>1950]
sea_level2= sea_level2[sea_level['Year']<2019].reset_index()
sea_level2

Criando a coluna com o calculo da taxa da elevação do nivel do mar

In [ ]:
sea_level2[['Variação da elevação dos mares']] =  (sea_level2[['Elevação do nivel do mar(polegadas)']] - sea_level1[['Elevação do nivel do mar(polegadas)']])/sea_level1[['Elevação do nivel do mar(polegadas)']]

In [ ]:
sea_level2

Adicionando a taxa do nivel do mar ao dataframe com as informações dos EUA

In [ ]:
dataframe_USA_groupby_final['Variação da elevação dos mares']= sea_level2[['Variação da elevação dos mares']]

Plotando o gráfico comparando as taxas de crescimento de ataques e da elevação dos mares

In [ ]:
dataframe_USA_groupby_final.plot(x='Ano',y= ['Variação Ano a Ano dos Ataques de Tubarão','Variação da elevação dos mares']) 

In [ ]:
dataframe_USA_groupby_final.plot.scatter(x ='Variação Ano a Ano dos Ataques de Tubarão' ,y= 'Variação da elevação dos mares', xlim=(-1,1) , ylim=(-0.1,0.06) )

In [ ]:
dataframe_USA_groupby_final.corr()

Importando e limpando o dataframe com as informações da anomalia da temperatura do mar

In [ ]:
sea_temp = pd.read_csv('data/sea-surface-temp_fig-1.csv')
sea_temp = sea_temp[sea_temp['Year']>1950]
sea_temp = sea_temp[sea_temp['Year']<2019].reset_index()
sea_temp

Adicionando a anomalia da temperatura do mar ao dataframe com as informações dos EUA

In [ ]:
dataframe_USA_groupby_final['Anomalia temperatura dos mares']= sea_temp[['Annual anomaly']

Plotando o gráfico comparando as taxas de crescimento de ataques e anomalia

In [ ]:
dataframe_USA_groupby_final.plot(x='Ano',y= ['Variação Ano a Ano dos Ataques de Tubarão','Anomalia temperatura dos mares']) 

In [ ]:
dataframe_USA_groupby_final.plot.scatter(x ='Variação Ano a Ano dos Ataques de Tubarão' ,y= 'Anomalia temperatura dos mares', xlim=(-1,1) , ylim=(-0.6,0.5) )

# Exportando como .csv

In [ ]:
dataframe_USA_groupby_final.to_csv('ataque_tubarao_clean.csv', index=False)